In [2]:
!wget -O data/input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt 

--2023-08-29 14:27:42--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘data/input.txt’

data/input.txt      100%[===================>]   1.06M  3.35MB/s    in 0.3s    

2023-08-29 14:27:43 (3.35 MB/s) - ‘data/input.txt’ saved [1115394/1115394]



In [3]:
with open('data/input.txt') as f:
    text = f.read()

In [4]:
"Total characters:", len(text)

('Total characters:', 1115394)

In [5]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [7]:
vocab_size

65

In [8]:
"".join(chars)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [9]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[x] for x in s]
decode = lambda e: "".join([itos[x] for x in e])


_ = encode("hi there")

print("Encoded:", _)
print("Decoded:", decode(_))

Encoded: [46, 47, 1, 58, 46, 43, 56, 43]
Decoded: hi there


In [10]:
import torch

In [13]:
torch.long

torch.int64

In [12]:
data = torch.tensor(encode(text), dtype=torch.long)

/tmp/ipykernel_12022/4070611439.py:1: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  data = torch.tensor(encode(text), dtype=torch.long)


In [14]:
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [15]:
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

## Train test split

In [16]:
n = int(.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [17]:
len(train_data), len(val_data)

(1003854, 111540)

In [18]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [19]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


## Batching

In [47]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [48]:
# get_batch('train')

In [49]:
xb, yb = get_batch('train')

for b in range(batch_size):
    print(f"Batch: {b}")
    for t in range(block_size):
        context = xb[b][:t+1]
        target = yb[b][t]
        print(f"Context: {context} Target: {target}")

Batch: 0
Context: tensor([24]) Target: 43
Context: tensor([24, 43]) Target: 58
Context: tensor([24, 43, 58]) Target: 5
Context: tensor([24, 43, 58,  5]) Target: 57
Context: tensor([24, 43, 58,  5, 57]) Target: 1
Context: tensor([24, 43, 58,  5, 57,  1]) Target: 46
Context: tensor([24, 43, 58,  5, 57,  1, 46]) Target: 43
Context: tensor([24, 43, 58,  5, 57,  1, 46, 43]) Target: 39
Batch: 1
Context: tensor([44]) Target: 53
Context: tensor([44, 53]) Target: 56
Context: tensor([44, 53, 56]) Target: 1
Context: tensor([44, 53, 56,  1]) Target: 58
Context: tensor([44, 53, 56,  1, 58]) Target: 46
Context: tensor([44, 53, 56,  1, 58, 46]) Target: 39
Context: tensor([44, 53, 56,  1, 58, 46, 39]) Target: 58
Context: tensor([44, 53, 56,  1, 58, 46, 39, 58]) Target: 1
Batch: 2
Context: tensor([52]) Target: 58
Context: tensor([52, 58]) Target: 1
Context: tensor([52, 58,  1]) Target: 58
Context: tensor([52, 58,  1, 58]) Target: 46
Context: tensor([52, 58,  1, 58, 46]) Target: 39
Context: tensor([52, 

In [50]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [67]:
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size): 
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)    
            loss = F.cross_entropy(logits, targets)
                    
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for  _ in range(max_new_tokens):
            logits, loss = self(idx)

            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)

            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [68]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
logits.shape, loss.shape

(torch.Size([32, 65]), torch.Size([]))

In [69]:
loss

tensor(4.8786, grad_fn=<NllLossBackward0>)

In [70]:
idx = torch.zeros((1, 1), dtype=torch.long)
idx

tensor([[0]])

In [70]:
idx = torch.zeros((1, 1), dtype=torch.long)
idx

tensor([[0]])

In [71]:
[generated] = m.generate(idx, max_new_tokens=100)
decode(generated.tolist())

"\nSr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3"

## Optimize

In [73]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [88]:
batch_size = 32

for steps in range(1000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4988932609558105


In [92]:
[generated] = m.generate(idx, max_new_tokens=500)

print(
    decode(generated.tolist())
)


y
Yeplount;
AUKironongan tak t jst, wan, meriongry henowatsthepe ainceleand t, fig:
NELLYORComicecarindiss spoubloureroanrsth, wisshr ppprt nd,
My t atht ld hathexcoute:
ppe

LOvere ing beeses, u biue p! nofllealllatho hathe thowil oute lf ur,

Insihe wan hin bo tllaighelat at; b n asir cove, tis, tond d mall amid s
CUSouss hatou t
BRTome,

I Yere t t totly ngut angustculexproour whis misorabround.


Bul bety
Yond sithe, sthigaist tsoss s ss than ke how,
mowil ring ISSAUSqur ck d my w wmeart sca


## The mathematical trick in self attention

In [93]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [101]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, 0)

In [102]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [103]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [118]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdims=True)

In [119]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [120]:
xbow2 = wei @ x # (B,(T,T)) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [121]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [123]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

### Version 4: Self attention

In [130]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# Single attention head
head_size = 16

key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)


k = key(x)   # (B, T, head_size)
q = query(x) # (B, T, head_size)
v = value(x)

wei = q @ k.transpose(-2, -1) * (head_size ** -.5) # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei@v # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)

out.shape

torch.Size([4, 8, 16])

In [138]:
_ = torch.randn(5)
print(_)
print(F.softmax(_, dim=-1))
print(F.softmax(_*16, dim=-1))

tensor([ 0.7153, -0.1060,  0.5602, -0.0738,  0.9711])
tensor([0.2474, 0.1088, 0.2119, 0.1124, 0.3195])
tensor([1.6387e-02, 3.2160e-08, 1.3700e-03, 5.3898e-08, 9.8224e-01])
